<a href="https://colab.research.google.com/github/khushishelat/GenderBiasSportsJournalism/blob/Khushi/NMF_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we attempt topic modelling in another way - using NMF (Nonnegative Matrix Factorization).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn

In [9]:
pip install pyLDAvis.sklearn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ERROR: Could not find a version that satisfies the requirement pyLDAvis.sklearn (from versions: none)
ERROR: No matching distribution found for pyLDAvis.sklearn


In [10]:
import pandas as pd
import re, nltk, spacy, gensim, string

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize
from pprint import pprint

# Plotting tools
import pyLDAvis
#mport pyLDAvis.sklearn
import pyLDAvis.lda_model
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
questions_df_cleaned = pd.read_csv('/content/drive/MyDrive/CIS 5300 NLP Project/Project Notebook/Data/tennis_data/questions_df_cleaned.csv')

In [11]:
n_features = 4000
n_components = 20
n_top_words = 20

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# ignore terms that have a document frequency strictly higher than 95%,
# ignore terms that have a document frequency strictly lower than 2
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(questions_df_cleaned['questions'].values.astype(str))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
nmf = NMF(n_components=n_components, random_state=1, l1_ratio=.5).fit(tfidf)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
nmf_output = nmf.fit_transform(tfidf)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
import numpy as np

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
def show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=tfidf_vectorizer, lda_model=nmf, n_words=20)

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,play,difficult,doubles,tennis,didn,level,tomorrow,players,week,player,cup,better,best,round,able,seen,want,just,said,serena
Topic 1,did,say,come,watch,make,change,affect,start,night,tell,crowd,expect,learn,help,coach,end,surprise,time,notice,point
Topic 2,like,looked,kind,player,really,just,big,lot,things,players,looks,felt,look,sort,don,crowd,people,guy,serve,tennis
Topic 3,match,thoughts,point,tough,talk,points,end,just,remember,went,watch,tonight,tomorrow,night,kind,close,lost,round,won,difficult
Topic 4,today,performance,serve,difference,happy,pleased,conditions,tough,didn,win,opponent,yesterday,day,went,way,happened,surprised,heat,crowd,wrong
Topic 5,set,second,happened,point,serve,points,end,break,got,won,lost,really,fifth,looked,went,came,tiebreak,fourth,didn,start
Topic 6,think,better,ll,players,don,best,tennis,player,people,maybe,far,beat,years,difference,need,opponent,win,andy,roger,federer
Topic 7,feel,better,pressure,physically,way,right,does,comfortable,different,coming,confident,getting,level,tired,crowd,moment,need,mentally,round,stronger
Topic 8,going,mind,week,open,forward,tomorrow,time,final,wimbledon,just,things,head,point,thought,tonight,said,try,season,cup,sort
Topic 9,year,open,better,wimbledon,end,different,season,compared,ago,goals,final,won,time,lot,maybe,years,rest,obviously,ve,coming


## Male DF NMF Topics

In [23]:
questions_df_cleaned_male = questions_df_cleaned[questions_df_cleaned['gender'] == 'M']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
questions_df_cleaned_female = questions_df_cleaned[questions_df_cleaned['gender'] == 'F']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
# ignore terms that have a document frequency strictly higher than 95%,
# ignore terms that have a document frequency strictly lower than 2
tfidf_vectorizer_male = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf_male = tfidf_vectorizer_male.fit_transform(questions_df_cleaned_male['questions'].values.astype(str))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
nmf_male = NMF(n_components=n_components, random_state=1, l1_ratio=.5).fit(tfidf_male)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
nmf_output_male = nmf_male.fit_transform(tfidf_male)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
topic_keywords_male = show_topics(vectorizer=tfidf_vectorizer_male, lda_model=nmf_male, n_words=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# Topic - Keywords Dataframe
df_topic_keywords_male = pd.DataFrame(topic_keywords_male)
df_topic_keywords_male.columns = ['Word '+str(i) for i in range(df_topic_keywords_male.shape[1])]
df_topic_keywords_male.index = ['Topic '+str(i) for i in range(df_topic_keywords_male.shape[0])]
df_topic_keywords_male

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,think,better,players,win,tournament,ll,don,best,player,federer,andy,people,far,nadal,roger,tennis,murray,maybe,years,rafa
Topic 1,did,say,come,make,watch,night,change,affect,start,crowd,expect,learn,tell,help,end,tonight,yesterday,long,injury,point
Topic 2,like,looked,guy,player,really,kind,big,crowd,lot,players,just,looks,sort,people,serve,guys,matches,don,tournament,felt
Topic 3,match,thoughts,point,talk,tough,end,tonight,remember,points,just,tomorrow,watch,went,night,kind,roger,federer,close,said,round
Topic 4,today,performance,serve,difference,tough,pleased,conditions,happy,win,didn,yesterday,day,went,surprised,heat,lot,opponent,better,lost,serving
Topic 5,year,better,wimbledon,open,end,compared,different,final,ago,season,ve,goals,won,say,grand,tournament,lot,great,years,rest
Topic 6,set,second,serve,point,happened,points,break,fifth,fourth,end,got,tiebreak,won,lost,really,went,didn,sets,looked,tiebreaker
Topic 7,feel,better,pressure,physically,way,right,comfortable,coming,getting,confident,different,mentally,best,need,tired,said,crowd,stronger,moment,got
Topic 8,yes,just,doubles,roger,tell,years,ago,looking,maybe,want,stage,season,possible,saying,answer,proud,games,generally,generation,sick
Topic 9,play,difficult,doubles,tomorrow,tennis,player,level,cup,nadal,didn,best,andy,better,davis,players,federer,week,expect,style,murray


## Female NMF Topics

In [31]:
# ignore terms that have a document frequency strictly higher than 95%,
# ignore terms that have a document frequency strictly lower than 2
tfidf_vectorizer_female = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf_female = tfidf_vectorizer_female.fit_transform(questions_df_cleaned_female['questions'].values.astype(str))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# alpha=0 means no regularization, l1_ratio=.5, the penalty is a combination of L1 and L2
nmf_female = NMF(n_components=n_components, random_state=1, l1_ratio=.5).fit(tfidf_female)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
nmf_output_female = nmf_female.fit_transform(tfidf_female)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
topic_keywords_female = show_topics(vectorizer=tfidf_vectorizer_female, lda_model=nmf_female, n_words=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# Topic - Keywords Dataframe
df_topic_keywords_female = pd.DataFrame(topic_keywords_female)
df_topic_keywords_female.columns = ['Word '+str(i) for i in range(df_topic_keywords_female.shape[1])]
df_topic_keywords_female.index = ['Topic '+str(i) for i in range(df_topic_keywords_female.shape[0])]
df_topic_keywords_female

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,win,tournament,slam,grand,big,won,best,open,important,far,winning,coming,final,title,wimbledon,career,matches,happy,obviously,players
Topic 1,did,say,come,court,coach,watch,affect,start,change,tell,help,crowd,surprise,make,night,learn,expect,notice,point,end
Topic 2,like,looked,court,kind,player,things,big,really,players,lot,look,felt,looks,said,don,getting,conditions,didn,sort,maybe
Topic 3,match,thoughts,tough,point,points,talk,end,went,watch,remember,tomorrow,tonight,serena,night,won,close,round,lost,long,nervous
Topic 4,set,second,happened,end,point,got,serve,won,lost,points,break,came,really,looked,thinking,able,went,start,love,mind
Topic 5,today,performance,difference,serve,happy,pleased,conditions,didn,opponent,way,happened,tough,played,day,yesterday,crowd,went,felt,feeling,wrong
Topic 6,think,ll,players,better,best,tennis,player,don,need,beat,women,difference,serena,maybe,point,people,final,years,come,opponent
Topic 7,feel,better,pressure,way,right,physically,different,level,comfortable,coming,confident,week,tired,moment,form,crowd,matches,need,best,final
Topic 8,going,mind,open,week,forward,tomorrow,wimbledon,point,head,feeling,final,confidence,obviously,sort,said,plan,strategy,thinking,confident,try
Topic 9,bit,little,talk,maybe,tell,tired,serve,nervous,kind,different,looked,getting,court,got,things,slow,felt,change,working,want
